In [3]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [4]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
228,You should not take what I am about to say lig...,positive
369,Guys and Dolls is a unique play based on the c...,positive
367,Probably the most accurate Stephen King adapti...,positive
184,"Narratives  whether written, visual or poetic...",positive
474,I really don't understand who this movie is ai...,negative


In [5]:
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:29: SyntaxWarning: invalid escape sequence '\s'
<>:29: SyntaxWarning: invalid escape sequence '\s'
/var/folders/c7/1v5wlgds2rjfw4_ldh17bj3h0000gn/T/ipykernel_34579/3773180380.py:29: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [6]:
df = normalize_text(df)
df.head()

,review,sentiment
228,take say lightly i ve seen many many film revi...,positive
369,guy doll unique play based character sky maste...,positive
367,probably accurate stephen king adaption yet su...,positive
184,narrative  whether written visual poetic epic...,positive
474,really understand movie aimed at absurdity it ...,negative


In [7]:
df['sentiment'].value_counts()

sentiment
negative    271
positive    229
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].map({'positive': 1, 'negative': 0})
df.head()

,review,sentiment
228,take say lightly i ve seen many many film revi...,1
369,guy doll unique play based character sky maste...,1
367,probably accurate stephen king adaption yet su...,1
184,narrative  whether written visual poetic epic...,1
474,really understand movie aimed at absurdity it ...,0


In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [21]:
vectorizer = CountVectorizer(max_features=50)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [23]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/princekhunt/TrackerX-MLOps-Experiment.mlflow')
dagshub.init(repo_owner='princekhunt', repo_name='TrackerX-MLOps-Experiment', mlflow=True)

mlflow.set_experiment("Logistic Regression Baseline")

2025-04-14 14:22:14,752 - INFO - HTTP Request: GET https://dagshub.com/api/v1/repos/princekhunt/TrackerX-MLOps-Experiment "HTTP/1.1 200 OK"


Initialized MLflow to track repo "princekhunt/TrackerX-MLOps-Experiment"

2025-04-14 14:22:14,763 - INFO - Initialized MLflow to track repo "princekhunt/TrackerX-MLOps-Experiment"


Repository princekhunt/TrackerX-MLOps-Experiment initialized!

2025-04-14 14:22:14,765 - INFO - Repository princekhunt/TrackerX-MLOps-Experiment initialized!


<Experiment: artifact_location='mlflow-artifacts:/d71f7b78cef3473b87b470988fae39da', creation_time=1744620248137, experiment_id='0', last_update_time=1744620248137, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [ ]:
import mlflow
import logging
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 50)
        mlflow.log_param("test_size", 0.20)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=5000) 

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)

2025-04-14 14:22:15,774 - INFO - Starting MLflow run...
2025-04-14 14:22:16,257 - INFO - Logging preprocessing parameters...
2025-04-14 14:22:18,258 - INFO - Initializing Logistic Regression model...
2025-04-14 14:22:18,258 - INFO - Fitting the model...
2025-04-14 14:22:18,270 - INFO - Model training complete.
2025-04-14 14:22:18,271 - INFO - Logging model parameters...
2025-04-14 14:22:18,771 - INFO - Making predictions...
2025-04-14 14:22:18,773 - INFO - Calculating evaluation metrics...
2025-04-14 14:22:18,783 - INFO - Logging evaluation metrics...
2025-04-14 14:22:20,614 - INFO - Saving and logging the model...
2025/04/14 14:22:22 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025-04-14 14:22:26,765 - INFO - Model training and logging completed in 10.51 seconds.
2025-04-14 14:22:26,766 - INFO - Accuracy: 0.52
2025-04-14 14:22:26,766 - INFO - Precision: 0

🏃 View run luminous-kit-415 at: https://dagshub.com/princekhunt/TrackerX-MLOps-Experiment.mlflow/#/experiments/0/runs/2aba10fb184742c78798118f6147b828
🧪 View experiment at: https://dagshub.com/princekhunt/TrackerX-MLOps-Experiment.mlflow/#/experiments/0
